In [1]:
%load_ext autoreload
%autoreload 2

# Load model

In [2]:
from pointnet2.models.pointnet2_msg_sem import Pointnet2MSG
import torch.nn as nn

model = Pointnet2MSG(num_classes=13, input_channels=0, use_xyz=True)
model.cuda()

Pointnet2MSG(
  (SA_modules): ModuleList(
    (0): PointnetSAModuleMSG(
      (groupers): ModuleList(
        (0): QueryAndGroup()
        (1): QueryAndGroup()
      )
      (mlps): ModuleList(
        (0): SharedMLP(
          (layer0): Conv2d(
            (conv): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer1): Conv2d(
            (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer2): Conv2d(
            (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
       

In [3]:
import etw_pytorch_utils.pytorch_utils as pt_utils
import torch

checkpoint_name = "checkpoints/poitnet2_semseg_best.pth.tar"
checkpoint = torch.load(checkpoint_name)
model.load_state_dict(checkpoint["model_state"])
model.cuda()

Pointnet2MSG(
  (SA_modules): ModuleList(
    (0): PointnetSAModuleMSG(
      (groupers): ModuleList(
        (0): QueryAndGroup()
        (1): QueryAndGroup()
      )
      (mlps): ModuleList(
        (0): SharedMLP(
          (layer0): Conv2d(
            (conv): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer1): Conv2d(
            (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer2): Conv2d(
            (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
       

In [4]:
import torch.nn as nn
from pointnet2.models.pointnet2_msg_sem import model_fn_decorator

model_fn = model_fn_decorator(nn.CrossEntropyLoss())

In [5]:
dist = compute_vec_dist(f1, f2)
print(dist)

NameError: name 'compute_vec_dist' is not defined

# Load my point cloud

In [6]:
import numpy as np
from numpy import genfromtxt
g = genfromtxt("/home/ryan/Desktop/PointClouds/global-using.txt", delimiter=",", dtype=np.float32)
l = genfromtxt("/home/ryan/Desktop/PointClouds/live-using.txt", delimiter=",", dtype=np.float32)

print(g.shape, l.shape)

(79853, 3) (69180, 3)


In [7]:
# sample_interval = 100
# g = g[range(0, g.shape[0], sample_interval), :]
# l = l[range(0, l.shape[0], sample_interval), :]
# print(g.shape, l.shape)

In [8]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure()
ax = Axes3D(fig)

data_to_draw = l

sequence_containing_x_vals = data_to_draw[:, 0]
sequence_containing_y_vals = data_to_draw[:, 1]
sequence_containing_z_vals = data_to_draw[:, 2]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [9]:
import numpy as np
import torch
B = 1
N_g = g.shape[0]
N_l = l.shape[0]

g_tor = torch.from_numpy(g).unsqueeze(0).cuda()
l_tor = torch.from_numpy(l).unsqueeze(0).cuda()

labels_g = torch.from_numpy(np.zeros((1, B * N_g), dtype = int)).view(B, N_g).cuda()
labels_l = torch.from_numpy(np.zeros((1, B * N_l), dtype = int)).view(B, N_l).cuda()

g_data = (g_tor, labels_g)
l_data = (l_tor, labels_l)

In [10]:
print(l_tor.shape, labels_l.shape)
print(g_tor.shape, labels_g.shape)

torch.Size([1, 69180, 3]) torch.Size([1, 69180])
torch.Size([1, 79853, 3]) torch.Size([1, 79853])


## NN extracts features

In [11]:
_, _, _ = model_fn(model, g_data, eval=True)
fg = model.get_features()

_, _, _ = model_fn(model, l_data, eval=True)
fl = model.get_features()

fg = fg.squeeze().transpose(1, 0)
fl = fl.squeeze().transpose(1, 0)

print(fg.shape)
print(fl.shape)

torch.Size([79853, 128])
torch.Size([69180, 128])


## BF matching 

In [12]:
# def compute_vec_dist_batch(f1, f2):
#     vec_diff = (f1 - f2)
#     sums = torch.sum(torch.pow(vec_diff, 2.0), dim=2)
#     res = torch.sqrt(sums)
#     return res

In [13]:
# # list of (idx, matching dist), 
# matches = torch.empty(size=(len(g), 2), dtype=torch.float32)
# dist = compute_vec_dist(fg.unsqueeze(1), fl.unsqueeze(0))

In [14]:
def compute_vec_feature_dist(f1, f2):
    vec_diff = (f1 - f2)
    sums = torch.sum(torch.pow(vec_diff, 2.0), dim=1)
    res = torch.sqrt(sums)
    return res

In [15]:
def compute_vec_geo_feature_dist(f1, f2):
    vec_diff = (f1 - f2)
    res = torch.pow(vec_diff, 2.0)
    geo_dist = torch.sqrt(torch.sum(res[:, :3], dim=1))
    fea_dist = torch.sqrt(torch.sum(res[:, 3:], dim=1))
    return geo_dist, fea_dist

In [16]:
def geo_dist_filter(geo_dist, fea_dist):
    res = torch.where(geo_dist < 0.02, fea_dist, torch.tensor(float('inf')).cuda())
    return res

In [17]:
# concat a point xyz with it's feature so it become 3 + 128 elements for each point
xyz_fl = torch.cat((l_tor.squeeze(), fl), dim=1)
xyz_fg = torch.cat((g_tor.squeeze(), fg), dim=1)
print(xyz_fl.shape)

torch.Size([69180, 131])


In [18]:
# find a match for every local point
matches = torch.zeros(size=(len(l), 3), dtype=torch.float32)
for i, f1 in enumerate(xyz_fl):
    if i%5000==0: print(i)
        
    geo_dist, fea_dist = compute_vec_geo_feature_dist(f1, xyz_fg)   
    fea_dist = geo_dist_filter(geo_dist, fea_dist)
    min_fea_dist, idx = torch.min(fea_dist, dim=0)
    matches[i] = torch.tensor([idx, min_fea_dist, geo_dist[idx]])

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000


In [19]:
print(matches)

tensor([[10.0000,  2.8199,  0.0196],
        [11.0000,  2.6071,  0.0198],
        [ 7.0000,  2.2826,  0.0191],
        ...,
        [ 0.0000,     inf,  0.4190],
        [ 0.0000,     inf,  0.4061],
        [ 0.0000,     inf,  0.4215]])


In [20]:
l[0] - g[6292]

array([-0.00785609, -0.06994038,  0.05911916], dtype=float32)

In [21]:
print(fea_dist.shape)
print(geo_dist.shape)

torch.Size([79853])
torch.Size([79853])


## Compute vector field

In [22]:
vec = g[matches[:, 0].int().tolist()] - l

print(vec.shape)

vec_mag = matches[:, 2]
print(vec_mag)
for i, v_mag in enumerate(vec_mag):
    
    # do not show if vector is too large. this can happen when the best feature matching distance is +inf..
    if v_mag > 0.1: vec[i] = [0,0,0]

# for i, v in enumerate(vec):
#     if i > 0:
#         vec[i] = [0,0,0]
        
print(vec[0])
print(vec)

(69180, 3)
tensor([0.0196, 0.0198, 0.0191,  ..., 0.4190, 0.4061, 0.4215])
[ 0.019382   -0.00091868 -0.00256443]
[[ 0.019382   -0.00091868 -0.00256443]
 [ 0.01935392 -0.00075515 -0.00416476]
 [ 0.01686193 -0.00359241  0.00817555]
 ...
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


### Write the vector field to txt

In [102]:
np.savetxt(fname="/home/ryan/Desktop/PointClouds/vec.txt", X=vec, delimiter=",")